# Necessary Imports and Installs

In [1]:
import sklearn as sk
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
%pip install freegs -q

Note: you may need to restart the kernel to use updated packages.


# Playground For Testing